# Magnetotellurics

This object can be used to store magnetotelluric (MT) surveys - a natural-source geophysical method. Data are provided in the frequency-domain as point source measurements of either impedances or apparent resistity/phase.

The following example shows how to generate an MT survey with associated data stored in `geoh5` format and accessible from [Geoscience ANALYST](https://mirageoscience.com/mining-industry-software/geoscience-analyst/).


![mtSurvey](./images/mt_survey.png)

In [1]:
import numpy as np
from geoh5py.workspace import Workspace
from geoh5py.objects import MTReceivers

# Create a new project
workspace = Workspace("my_project.geoh5")

# Define a synthetic survey with receivers on 2 lines, 60 m apart
x_loc, y_loc = np.meshgrid(np.linspace(-5, 5, 2), np.linspace(0., 20., 9))
vertices = np.c_[x_loc.ravel(), y_loc.ravel(), np.zeros_like(x_loc).ravel()]

# Create the survey from vertices
mt_survey = MTReceivers.create(workspace, vertices=vertices)

Only receivers are needed to define the survey as MT uses the ambient electromagntic field of the Earth - no transmitters (source) required.

## Metadata

Along with the [MTReceivers](../api/geoh5py.objects.surveys.electromagnetics.rst#module-geoh5py.objects.surveys.electromagnetics.magnetotellurics), the metadata contains all the necessary information to define the geophysical experiment.

In [2]:
mt_survey.metadata

{'EM Dataset': {'Channels': [],
  'Input type': 'Rx only',
  'Property groups': [],
  'Receivers': UUID('c9751d22-22ca-4e03-bb74-eeb4b7c57d10'),
  'Survey type': 'Magnetotellurics',
  'Unit': 'Hertz (Hz)'}}

### Channels

List of frequencies at which the data are provided.

In [3]:
mt_survey.channels = [1., 10., 100.]

### Input type

Generic label used in the `geoh5` standard for all EM survey entities. Restricted to `Rx only` in the case of natural sources methods.

### Property groups

List of [PropertyGroup](../api/geoh5py.groups.rst#module-geoh5py.groups.property_group)s defining the various data components (e.g. `Zxx (real)`, `Zxy (imag)`, ...). It is not required to supply all components of the impedence tensor, but it is expected that each component contains a list of data channels of length and in the same order as the `Channels` (one `Data` per frequency).

The class method [add_components_data](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.base.BaseEMSurvey.add_components_data) can help users add data from nested dictionaries. Below is an example using four components:

In [4]:
# Arbitrary data generator using sine functions
data_fun = lambda c, f: (c+1.) * np.sin(f * np.pi * (x_loc * y_loc).ravel() / 200.)

# Create a nested dictionary of component and frequency data.
data = {
    component : {
        f"{component}_{freq}": {"values": (ff+1)*1000. + (cc+1) * 100. + np.arange(vertices.shape[0])} for ff, freq in enumerate(mt_survey.channels)
    } for cc, component in enumerate([
        "Zxx (real)", "Zxx (imaginary)",
        "Zxy (real)", "Zxy (imaginary)",
        "Zyx (real)", "Zyx (imaginary)",
        "Zyy (real)", "Zyy (imaginary)",
    ])
}
    
mt_survey.add_components_data(data)

Metadata are updated immediately to reflect the addition of components:

In [5]:
mt_survey.metadata

{'EM Dataset': {'Channels': [1.0, 10.0, 100.0],
  'Input type': 'Rx only',
  'Property groups': ['Zxx (real)',
   'Zxx (imaginary)',
   'Zxy (real)',
   'Zxy (imaginary)',
   'Zyx (real)',
   'Zyx (imaginary)',
   'Zyy (real)',
   'Zyy (imaginary)'],
  'Receivers': UUID('c9751d22-22ca-4e03-bb74-eeb4b7c57d10'),
  'Survey type': 'Magnetotellurics',
  'Unit': 'Hertz (Hz)'}}

Data channels associated with each component can be quickly accessed through the [BaseEMSurvey.components](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.base.BaseEMSurvey.components) property:

In [6]:
mt_survey.components

{'Zxx (real)': [<geoh5py.data.float_data.FloatData at 0x1984bec6050>,
 'Zxx (imaginary)': [<geoh5py.data.float_data.FloatData at 0x1984bec6440>,
 'Zxy (real)': [<geoh5py.data.float_data.FloatData at 0x1984bec6380>,
 'Zxy (imaginary)': [<geoh5py.data.float_data.FloatData at 0x1984bec7d00>,
 'Zyx (real)': [<geoh5py.data.float_data.FloatData at 0x1984bec4100>,
 'Zyx (imaginary)': [<geoh5py.data.float_data.FloatData at 0x1984bec4a60>,
 'Zyy (real)': [<geoh5py.data.float_data.FloatData at 0x1984bf7c970>,
 'Zyy (imaginary)': [<geoh5py.data.float_data.FloatData at 0x1984bf7d1b0>,
  <geoh5py.data.float_data.FloatData at 0x1984bf7d660>]}

### Receivers

Generic label used in the `geoh5` standard for EM survey to identify the receiver entity. Restricted to itself in the case of `MTReceivers`.

### Survey type

Label identifier for `Magnetotellurics` survey type.

### Unit

Units for frequency sampling of the data: `Hertz (Hz)`, `KiloHertz (kHz)`, `MegaHertz (MHz)` or `Gigahertz (GHz)`.

In [7]:
workspace.close()